# changing

https://www.kaggle.com/cristianossd/tf-idf-approach-on-insincere-questions

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk import word_tokenize
from scipy.sparse import coo_matrix
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))


In [ ]:
df = pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')#[:100000]
df.head()

In [ ]:
df['target'].value_counts()

### don't  Resample

Trying undersampling strategy:

### change tfidf to countvectorizer binary and use 90k features
use all words test + train
usually one uses all train words and omits all new trst words

In [ ]:



tf_vectorizer =CountVectorizer(binary=True,strip_accents='unicode',max_features=90000).fit(df['question_text'].append(test['question_text']))
listOfWords = tf_vectorizer.get_feature_names()
dictOfWords = { listOfWords[i]:i for i in range(0, len(listOfWords) ) }
tf_vectorizer.transform(df['question_text'])


**co-occurrance snippet**

In [ ]:
from nltk import word_tokenize
from scipy.sparse import coo_matrix

def create_cooccurrence_matrix(filename,tokenizer,window_size,vocabulary):
    #vocabulary={}
    data=[]
    row=[]
    col=[]
    for sentence in filename:
        sentence=sentence.strip()
        #print(sentence)
        tokens=[token for token in tokenizer(sentence) if token!=u""]
        for pos,token in enumerate(tokens):
            i=vocabulary.setdefault(token,len(vocabulary))
            start=max(0,pos-window_size)
            end=min(len(tokens),pos+window_size+1)
            for pos2 in range(start,end):
                if pos2==pos: 
                    continue
                j=vocabulary.setdefault(tokens[pos2],len(vocabulary))
                data.append(1.); row.append(i); col.append(j);
    
    cooccurrence_matrix=coo_matrix((data,(row,col)))
    return vocabulary,cooccurrence_matrix
#voca,coo=create_cooccurrence_matrix(df.question_text.str.lower().values,word_tokenize,100,dictOfWords)


**splitting and vectorizing**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer


X_train, X_test, y_train, y_test = train_test_split(df['question_text'],
                                                    df['target'],
                                                    test_size=0.2)
#tf_vectorizer = TfidfVectorizer().fit(df_under['question_text'])
#tf_vectorizer = CountVectorizer(tokenizer=word_tokenize,stop_words).fit(df['question_text'])
X_train = tf_vectorizer.transform(X_train)
X_test = tf_vectorizer.transform(X_test)
X_train.shape

In [ ]:
#from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(C=1.0,multi_class='multinomial',penalty='l2', solver='saga',n_jobs=-1)
clf.fit(X_train, y_train)
#clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)

In [ ]:
from sklearn.metrics import f1_score


f1_score(y_test, predicted,average=None)

### Submission dataset

In [ ]:
df_test = pd.read_csv('../input/test.csv')
X_submission = tf_vectorizer.transform(df_test['question_text'])
predicted_test = clf.predict(X_submission)

df_test['prediction'] = predicted_test
submission = df_test.drop(columns=['question_text'])
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)